<p>https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=11128</p>
<p>https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=[TaxId]</p>

In [ ]:
import re, os, json

import numpy as np

from Bio import Entrez

Entrez.email = "VIMVer@univ-amu.fr"

testydata: str = "./../../../testymolo/media/data/"
tables_csv: str = "./../../../testymolo/media/tables_csv/"

merge_json: str = "./../../../testymolo/media/data/"

In [ ]:
### caution when not all rows same nbr of col
def custom_csv_parser_to_list(infilepath:str):
    with open(infilepath ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            values.append(val)
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                values.append(val)

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        print("nbr of col", set([len(row) for row in rows]))
        print("nbr of rows", len(rows))
        return rows

In [ ]:
Organism_csv:list = custom_csv_parser_to_list(os.path.join(tables_csv, 'Organism.csv'))
Organism_h:list = ["Tax_id", "Name", "Categorie", "Classe", "Ordre", "Fam", "SsFam", "Genre", "Note_org"]

CAZy_DB_csv:list = custom_csv_parser_to_list(os.path.join(tables_csv, 'CAZy_DB.csv'))
CAZy_DB_h:list = ["DB_ac", "Protein", "DB_nom", "Organism", "abr", "Tax_id", "EC", "_3D_status",
                   "Length", "Sequence", "DB_note", "Created", "Modified", "PP_status", "Lib_sort"]


In [ ]:
def read_merge_data(infilepath:str):
    with open(infilepath, 'r') as handle :
        try:
            data = json.loads(handle.read())
        except:
            data = {}
            print("ERROR")
    return data

In [ ]:
Organism_json:list = read_merge_data(os.path.join(merge_json, 'Organism.temp.json'))

In [ ]:
for item in Organism_json:
    for key, value in item.items():
        print(key, value)
    break

In [ ]:
TaxId:int = 11120

print("Organism_csv")
for item in Organism_csv:
    if str(TaxId) == str(item[0]):
        print(item)
print("CAZy_DB_csv")
for item in CAZy_DB_csv:
    if str(TaxId) == str(item[5]):
        print(item)


In [ ]:
import subprocess as sh

#var = sh.run(['taxonkit', 'list', '--ids', '11120', '--indent', ''])
#  taxonkit list --show-rank --show-name --indent "    " --ids 11120 -J
var = sh.run(["taxonkit","list", "--show-rank", "--show-name", "--indent", "    ", "--ids", "2697049", "-J"], stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print(var.stdout)

In [ ]:

import json
var_json = json.loads(var.stdout)

def json2html(data:dict) -> str:
    #recursive
    txt:str = ""
    for line, subdata in data.items():
        line:str = line.strip()
        txt += "<li>"+line+"</li>"
        
        if len(subdata) > 0: #sub_clade
            txt += "<ul>" + json2html(subdata) + "</ul>"
    return txt

var_html = "<div class='taxonkit'><ul>"+json2html(var_json)+"</ul></div>"
print(var_html)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(var_html))

In [ ]:

display(HTML("<div><ul><li>species</li><ul><li>sub-species</li><li>sub-species</li></ul><li>species</li><li>species</li></ul></div>"))

In [ ]:
command = sh.run(['echo', 'Za Worudo !'], stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print(command.stdout)

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))


# NCBI genbank record

<ul>genome
    <li>ORF</li>
    <li>cds</li></ul>
<ul>proteome
    <li>protein</li></ul>

<ul>
<li>scr</li>
<li>view</li>
<li>url</li>
<li>js</li>
<li>html</li></ul>

In [ ]:
#does it work every time

Protein_json:list = read_merge_data(os.path.join(merge_json, 'Protein.temp.json'))

In [ ]:
for org in Organism_json:
    taxid = str(org['id'])
    proteins:list = [ prot for prot in Protein_json if str(prot['organism']) == taxid ]
    for prot in proteins:
        terms:str = f"{org['name']}[Organism] AND {prot['name']} AND refseq[filter]"
        print(terms)
        with Entrez.esearch(db='nucleotide', term=terms, idtype='acc') as handle :
            record = Entrez.read(handle)
            print(record['IdList'])

In [ ]:
from Bio import Entrez

Entrez.email = "VIMVer@univ-amu.fr"
terms:str = "txid2732396[Organism:exp] AND viruses[filter] AND refseq[filter]"

eresult = Entrez.esearch(db='protein', term=terms, idtype='acc')
bresult = Entrez.read(eresult)
print(bresult)
with open('./var', 'w') as handle:
    for record in bresult:
        print(record)
        #handle.write(record)
        
        break


In [ ]:
from Bio import Entrez
Entrez.email = "VIMVer@univ-amu.fr"
terms:str = "txid76804[Organism:exp]"
# 255 genomes

eresult = Entrez.esearch(db='genome', term=terms)
bresult = Entrez.read(eresult)
print(bresult)
for record in bresult:
    print(record)
    break

In [ ]:

# get complementary info on each

for accNumber in record['IdList']:
    print(accNumber)
    genbank = Entrez.efetch(db='nucleotide', id=accNumber, rettype='gb', retmode='text')
    print(genbank.readlines()[0].strip())


In [ ]:
# run blast
#blastp -query ./protein_test.fasta -db refseq_protein -max_target_seqs 5 -remote -outfmt 5 > var.blastp.html

import subprocess as sh

command_line = ['blastp', '-query', './protein_test.fasta', '-db', 'refseq_protein', '-max_target_seqs', '10', '-remote', '-outfmt', '5', '-out', './var.temp.blastp.xml']
command_line = sh.run(command_line, stdout=sh.PIPE, stderr=sh.PIPE, text=True)
print('uwu', command_line.stdout)


In [ ]:
import Bio.Blast.NCBIXML as BlastXML

with open("./var.temp.blastp.xml") as blastp_result:
    result = BlastXML.parse(blastp_result)

    for record in result:
        print(record)

In [ ]:
for record in result:
        print(record['descriptions'][0])

<p>Diamond</p>

In [ ]:
import subprocess as sh
import os
import Bio.Blast.NCBIXML as BlastXML


terms = "diamond blastp -d ./viral_protein_db/Nidovirales/nido -q ./protein_test.fasta -o ./var_4 -f 5"
terms = terms.split(' ')

diamond_run = sh.run(terms, stderr=sh.PIPE, stdout=sh.PIPE, text=True)
print("complete")
       

In [ ]:
with open("./var_4") as blastp_result:
    result = BlastXML.parse(blastp_result)

    for record in result:
        for hit in record.alignments:
            id = hit.hit_id
            definition = hit.hit_def
            start = hit.hsps[0].sbjct_start
            end = hit.hsps[0].sbjct_end
            align_length = hit.hsps[0].align_length
            identities = hit.hsps[0].identities
            _identities = str('{:10.2f}'.format(identities*100.0/align_length)).strip()
            positives = hit.hsps[0].positives
            _positives = str('{:10.2f}'.format(positives*100.0/align_length)).strip()
            bits = hit.hsps[0].bits

            print(id+'_'+str(start)+':'+str(end))
            print(f"{align_length} {identities}({_identities}%) {positives}({_positives}%) {bits}")
            print(definition)
            #break
            

In [ ]:
    
>ORF1ab_polyprotein__(@473)
MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEECDTNSECEEEDEDTKVLALIQDPASIKYPLPLDEDYSVYNGCIVHKDALDVVNLPSGEETFVVNNCFEGAVKPLPQKVVDVLGDWGEAVDAQEQLCQQEPLQHTFEEPVENSTGSSKTMTEQVVVEDQELPVVEQDQDVVVYTPTDLEVAKETAEEVDEFILIFAVPKEEVVSQKDGAQIKQEPIQVVKPQREKKAKKFKVKPATCEKPKFLEYKTCVGDLTVVIAKALDEFKEFCIVNAANEHMTHGSGVAKAIADFCGLDFVEYCEDYVKKHGPQQRLVTPSFVKGIQCVNNVVGPRHGDNNLHEKLVAAYKNVLVDGVVNYVVPVLSLGIFGVDFKMSIDAMREAFEGCTIRVLLFSLSQEHIDYFDVTCKQKTIYLTEDGVKYRSIVLKPGDSLGQFGQVYAKNKIVFTADDVEDKEILYVPTTDKSILEYYGLDAQKYVIYLQTLAQKWNVQYRDNFLILEWRDGNCWISSAIVLLQAAKIRFKGFLTEAWAKLLGGDPTDFVAWCYASCTAKVGDFSDANWLLANLAEHFDADYTNAFLKKRVSCNCGIKSYELRGLEACIQPVRATNLLHFKTQYSNCPTCGANNTDEVIEASLPYLLLFATDGPATVDCDEDAVGTVVFVGSTNSGHCYTQAAGQAFDNLAKDRKFGKKSPYITAMYTRFAFKNETSLPVAKQSKGKSKSVKEDVSNLATSSKASFDNLTDFEQWYDSNIYESLKVQESPDNFDKYVSFTTKEDSKLPLTLKVRGIKSVVDFRSKDGFIYKLTPDTDENSKAPVYYPVLDAISLKAIWVEGNANFVVGHPNYYSKSLHIPTFWENAENFVKMGDKIGGVTMGLWRAEHLNKPNLERIFNIAKKAIVGSSVVTTQCGKLIGKAATFIADKVGGGVVRNITDSIKGLCGITRGHFERKMSPQFLKTLMFFLFYFLKASVKSVVASYKTVLCKVVLATLLIVWFVYTSNPVMFTGIRVLDFLFEGSLCGPYKDYGKDSFDVLRYCADDFICRVCLHDKDSLHLYKHAYSVEQVYKDAASGFIFNWNWLYLVFLILFVKPVAGFVIICYCVKYLVLNSTVLQTGVCFLDWFVQTVFSHFNFMGAGFYFWLFYKIYIQVHHILYCKDVTCEVCKRVARSNRQEVSVVVGGRKQIVHVYTNSGYNFCKRHNWYCRNCDDYGHQNTFMSPEVAGELSEKLKRHVKPTAYAYHVVDEACLVDDFVNLKYKAATPGKDSASSAVKCFSVTDFLKKAVFLKEALKCEQISNDGFIVCNTQSAHALEEAKNAAIYYAQYLCKPILILDQALYEQLVVEPVSKSVIDKVCSILSSIISVDTAALNYKAGTLRDALLSITKDEEAVDMAIFCHNHDVDYTGDGFTNVIPSYGIDTGKLTPRDRGFLINADASIANLRVKNAPPVVWKFSELIKLSDSCLKYLISATVKSGVRFFITKSGAKQVIACHTQKLLVEKKAGGIVSGTFKCFKSYFKWLLIFYILFTACCSGYYYMEVSKSFVHPMYDVNSTLHVEGFKVIDKGVLREIVPEDTCFSNKFVNFDAFWGRPYDNSRNCPIVTAVIDGDGTVATGVPGFVSWVMDGVMFIHMTQTERKPWYIPTWFNREIVGYTQDSIITEGSFYTSIALFSARCLYLTASNTPQLYCFNGDNDAPGALPFGSIIPHRVYFQPNGVRLIVPQQILHTPYVVKFVSDSYCRGSVCEYTRPGYCVSLNPQWVLFNDEYTSKPGVFCGSTVRELMFSMVSTFFTGVNPNIYMQLATMFLILVVVVLIFAMVIKFQGVFKAYATTVFITMLVWVINAFILCVHSYNSVLAVILLVLYCYASLVTSRNTVIIMHCWLVFTFGLIVPTWLACCYLGFIIYMYTPLFLWCYGTTKNTRKLYDGNEFVGNYDLAAKSTFVIRGSEFVKLTNEIGDKFEAYLSAYARLKYYSGTGSEQDYLQACRAWLAYALDQYRNSGVEIVYTPPRYSIGVSRLQSGFKKLVSPSSAVEKCIVSVSYRGNNLNGLWLGDTIYCPRHVLGKFSGDQWNDVLNLANNHEFEVTTQHGVTLNVVSRRLKGAVLILQTAVANAETPKYKFIKANCGDSFTIACAYGGTVVGLYPVTMRSNGTIRASFLAGACGSVGFNIEKGVVNFFYMHHLELPNALHTGTDLMGEFYGGYVDEEVAQRVPPDNLVTNNIVAWLYAAIISVKESSFSLPKWLESTTVSVDDYNKWAGDNGFTPFSTSTAITKLSAITGVDVCKLLRTIMVKNSQWGGDPILGQYNFEDELTPESVFNQIGGVRLQSSFVRKATSWFWSRCVLACFLFVLCAIVLFTAVPLKFYVYAAVILLMAVLFISFTVKHVMAYMDTFLLPTLITVIIGVCAEVPFIYNTLISQVVIFLSQWYDPVVFDTMVPWMFLPLVLYTAFKCVQGCYMNSFNTSLLMLYQFVKLGFVIYTSSNTLTAYTEGNWELFFELVHTTVLANVSSNSLIGLFVFKCAKWMLYYCNATYLNNYVLMAVMVNCIGWLCTCYFGLYWWVNKVFGLTLGKYNFKVSVDQYRYMCLHKINPPKTVWEVFSTNILIQGIGGDRVLPIATVQAKLSDVKCTTVVLMQLLTKLNVEANSKMHVYLVELHNKILASDDVGECMDNLLGMLITLFCIDSTIDLSEYCDDILKRSTVLQSVTQEFSHIPSYAEYERAKNLYEKVLVDSKNGGVTQQELAAYRKAANIAKSVFDRDLAVQKKLDSMAERAMTTMYKEARVTDRRAKLVSSLHALLFSMLKKIDSEKLNVLFDQASSGVVPLATVPIVCSNKLTLVIPDPETWVKCVEGVHVTYSTVVWNIDTVIDADGTELHPTSTGSGLTYCISGANIAWPLKVNLTRNGHNKVDVVLQNNELMPHGVKTKACVAGVDQAHCSVESKCYYTNISGNSVVAAITSSNPNLKVASFLNEAGNQIYVDLDPPCKFGMKVGVKVEVVYLYFIKNTRSIVRGMVLGAISNVVVLQSKGHETEEVDAVGILSLCSFAVDPADTYCKYVAAGNQPLGNCVKMLTVHNGSGFAITSKPSPTPDQDSYGGASVCLYCRAHIAHPGSVGNLDGRCQFKGSFVQIPTTEKDPVGFCLRNKVCTVCQCWIGYGCQCDSLRQPKSSVQSVAGASDFDKNYLNRVRGSSEARLIPLASGCDPDVVKRAFDVCNKESAGMFQNLKRNCARFQELRDTEDGNLEYLDSYFVVKQTTPSNYEHEKSCYEDLKSEVTADHDFFVFNKNIYNISRQRLTKYTMMDFCYALRHFDPKDCEVLKEILVTYGCIEDYHPKWFEENKDWYDPIENSKYYVMLAKMGPIVRRALLNAIEFGNLMVEKGYVGVITLDNQDLNGKFYDFGDFQKTAPGAGVPVFDTYYSYMMPIIAMTDALAPERYFEYDVHKGYKSYDLLKYDYTEEKQELFQKYFKYWDQEYHPNCRDCSDDRCLIHCANFNILFSTLIPQTSFGNLCRKVFVDGVPFIATCGYHSKELGVIMNQDNTMSFSKMGLSQLMQFVGDPALLVGTSNNLVDLRTSCFSVCALTSGITHQTVKPGHFNKDFYDFAEKAGMFKEGSSIPLKHFFYPQTGNAAINDYDYYRYNRPTMFDICQLLFCLEVTSKYFECYEGGCIPASQVVVNNLDKSAGYPFNKFGKARLYYEMSLEEQDQLFEITKKNVLPTITQMNLKYAISAKNRARTVAGVSILSTMTNRQFHQKILKSIVNTRNASVVIGTTKFYGGWDNMLRNLIQGVEDPILMGWDYPKCDRAMPNLLRIAASLVLARKHTNCCSWSERIYRLYNECAQVLSETVLATGGIYVKPGGTSSGDATTAYANSVFNIIQATSANVARLLSVITRDIVYDNIKSLQYELYQQVYRRVNFDPAFVEKFYSYLCKNFSLMILSDDGVVCYNNTLAKQGLVADISGFREVLYYQNNVFMADSKCWVEPDLEKGPHEFCSQHTMLVEVDGEPKYLPYPDPSRILGACVFVDDVDKTEPVAVMERYIALAIDAYPLVHHENEEYKKVFFVLLAYIRKLYQELSQNMLMDYSFVMDIDKGSKFWEQEFYENMYRAPTTLQSCGVCVVCNSQTILRCGNCIRKPFLCCKCCYDHVMHTDHKNVLSINPYICSQLGCGEADVTKLYLGGMSYFCGNHKPKLSIPLVSNGTVFGIYRANCAGSENVDDFNQLATTNWSIVEPYILANRCSDSLRRFAAETVKATEELHKQQFASAEVREVFSDRELILSWEPGKTRPPLNRNYVFTGYHFTRTSKVQLGDFTFEKGEGKDVVYYKATSTAKLSVGDIFVLTSHNVVSLVAPTLCPQQTFSRFVNLRPNVMVPECFVNNIPLYHLVGKQKRTTVQGPPGSGKSHFAIGLAVYFSSARVVFTACSHAAVDALCEKAFKFLKVDDCTRIVPQRTTVDCFSKFKANDTGKKYIFSTINALPEVSCDILLVDEVSMLTNYELSFINGKINYQYVVYVGDPAQLPAPRTLLNGSLSPKDYNVVTNLMVCVKPDIFLAKCYRCPKEIVDTVSTLVYDGKFIANNPESRECFKVIVNNGNSDVGHESGSAYNTTQLEFVKDFVCRNKQWREAIFISPYNAMNQRAYRMLGLNVQTVDSSQGSEYDYVIFCVTADSQHALNINRFNVALTRAKRGILVVMRQRDELYSALKFTELDSETSLQGTGLFKICNKEFSGVHPAYAVTTKALAATYKVNDELAALVNVEAGSEITYKHLISLLGFKMSVNVEGCHNMFITRDEAIRNVRGWVGFDVEATHACGTNIGTNLPFQVGFSTGADFVVTPEGLVDTSIGNNFEPVNSKAPPGEQFNHLRVLFKSAKPWHVIRPRIVQMLADNLCNVSDCVVFVTWCHGLELTTLRYFVKIGKEQVCSCGSRATTFNSHTQAYACWKHCLGFDFVYNPLLVDIQQWGYSGNLQFNHDLHCNVHGHAHVASVDAIMTRCLAINNAFCQDVNWDLTYPHIANEDEVNSSCRYLQRMYLNACVDALKVNVVYDIGNPKGIKCVRRGDVNFRFYDKNPIVRNVKQFEYDYNQHKDKFADGLCMFWNCNVDCYPDNSLVCRYDTRNLSVFNLPGCNGGSLYVNKHAFYTPKFDRISFRNLKAMPFFFYDSSPCETIQVDGVAQDLVSLATKDCITKCNIGGAVCKKHAQMYAEFVTSYNAAVTAGFTFWVTNKLNPYNLWKSFSALQSIDNIAYNMYKGGHYDAIAGEMPTVITGDKVFVIDQGVEKAVFVNQTTLPTSVAFELYAKRNIRTLPNNRILKGLGVDVTNGFVIWDYANQTPLYRNTVKVCAYTDIEPNGLVVLYDDRYGDYQSFLAADNAVLVSTQCYKRYSYVEIPSNLLVQNGMPLKDGANLYVYKRVNGAFVTLPNTINTQGRSYETFEPRSDIERDFLAMSEESFVERYGKDLGLQHILYGEVDKPQLGGLHTVIGMYRLLRANKLNAKSVTNSDSDVMQNYFVLSDNGSYKQVCTVVDLLLDDFLELLRNILKEYGTNKSKVVTVSIDYHSINFMTWFEDGSIKTCYPQLQSAWTCGYNMPELYKVQNCVMEPCNIPNYGVGITLPSGILMNVAKYTQLCQYLSKTTICVPHNMRVMHFGAGSDKGVAPGSTVLKQWLPEGTLLVDNDIVDYVSDAHVSVLSDCNKYNTEHKFDLVISDMYTDNDSKRKHEGVIANNGNDDVFIYLSSFLRNNLALGGSFAVKVTETSWHEVLYDIAQDCAWWTMFCTAVNASSSEAFLIGVNYLGASEKVKVSGKTLHANYIFWRNCNYLQTSAYSIFDVAKFDLRLKATPVVNLKTEQKTDLVFNLIKCGKLLVRDVGNTSFTSDSFVCTM


In [ ]:
from datetime import datetime
print(datetime.now().strftime("%Y-%m-%d"))


In [ ]:
value = '70.01'
print( float(value) >
 70)

In [ ]:
# if sequence has changed
## header           -> aucune incidence 
## mut : S3M        -> correction
## indel : ILV16-   -> frame ajustment
_fasta:str = """>query
MAMSLKQGVSPKPRDVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEECDTNSECEEEDEDTKVLALIQDPASIKYPLPLDEDYSVYNGCIVHKDALDVVNLPSGEETFVVNNCFEGAVKPLPQKVVDVLGDWGEAVDAQEQLCQQEPLQHTFEEPVENSTGSSKTMTEQVVVEDQELPVVEQDQDVVVYTPTDLEVAKETAEEVDEFILIFAVPKEEVVSQKDGAQIKQEPIQVVKPQREKKAKKFKVKPATCEKPKFLEYKTCVGDLTVVIAKALDEFKEFCIVNAANEHMTHGSGVAKAIADFCGLDFVEYCEDYVKKHGPQQRLVTPSFVKGIQCVNNVVGPRHGDNNLHEKLVAAYKNVLVDGVVNYVVPVLSLGIFGVDFKMSIDAMREAFEGCTIRVLLFSLSQEHIDYFDVTCKQKTIYLTEDGVKYRSIVLKPGDSLGQFGQVYAKNKIVFTADDVEDKEILYVPTTDKSILEYYGLDAQKYVIYLQTLAQKWNVQYRDNFLILEWRDGNCWISSAIVLLQAAKIRFKGFLTEAWAKLLGGDPTDFVAWCYASCTAKVGDFSDANWLLANLAEHFDADYTNAFLKKRVSCNCGIKSYELRGLEACIQPVRATNLLHFKTQYSNCPTCGANNTDEVIEASLPYLLLFATDGPATVDCDEDAVGTVVFVGSTNSGHCYTQAAGQAFDNLAKDRKFGKKSPYITAMYTRFAFKNETSLPVAKQSKGKSKSVKEDVSNLATSSKASFDNLTDFEQWYDSNIYESLKVQESPDNFDKYVSFTTKEDSKLPLTLKVRGIKSVVDFRSKDGFIYKLTPDTDENSKAPVYYPVLDAISLKAIWVEGNANFVVGHPNYYSKSLHIPTFWENAENFVKMGDKIGGVTMGLWRAEHLNKPNLERIFNIAKKAIVGSSVVTTQCGKLIGKAATFIADKVGGGVVRNITDSIKGLCGITRGHFERKMSPQFLKTLMFFLFYFLKASVKSVVASYKTVLCKVVLATLLIVWFVYTSNPVMFTGIRVLDFLFEGSLCGPYKDYGKDSFDVLRYCADDFICRVCLHDKDSLHLYKHAYSVEQVYKDAASGFIFNWNWLYLVFLILFVKPVAGFVIICYCVKYLVLNSTVLQTGVCFLDWFVQTVFSHFNFMGAGFYFWLFYKIYIQVHHILYCKDVTCEVCKRVARSNRQEVSVVVGGRKQIVHVYTNSGYNFCKRHNWYCRNCDDYGHQNTFMSPEVAGELSEKLKRHVKPTAYAYHVVDEACLVDDFVNLKYKAATPGKDSASSAVKCFSVTDFLKKAVFLKEALKCEQISNDGFIVCNTQSAHALEEAKNAAIYYAQYLCKPILILDQALYEQLVVEPVSKSVIDKVCSILSSIISVDTAALNYKAGTLRDALLSITKDEEAVDMAIFCHNHDVDYTGDGFTNVIPSYGIDTGKLTPRDRGFLINADASIANLRVKNAPPVVWKFSELIKLSDSCLKYLISATVKSGVRFFITKSGAKQVIACHTQKLLVEKKAGGIVSGTFKCFKSYFKWLLIFYILFTACCSGYYYMEVSKSFVHPMYDVNSTLHVEGFKVIDKGVLREIVPEDTCFSNKFVNFDAFWGRPYDNSRNCPIVTAVIDGDGTVATGVPGFVSWVMDGVMFIHMTQTERKPWYIPTWFNREIVGYTQDSIITEGSFYTSIALFSARCLYLTASNTPQLYCFNGDNDAPGALPFGSIIPHRVYFQPNGVRLIVPQQILHTPYVVKFVSDSYCRGSVCEYTRPGYCVSLNPQWVLFNDEYTSKPGVFCGSTVRELMFSMVSTFFTGVNPNIYMQLATMFLILVVVVLIFAMVIKFQGVFKAYATTVFITMLVWVINAFILCVHSYNSVLAVILLVLYCYASLVTSRNTVIIMHCWLVFTFGLIVPTWLACCYLGFIIYMYTPLFLWCYGTTKNTRKLYDGNEFVGNYDLAAKSTFVIRGSEFVKLTNEIGDKFEAYLSAYARLKYYSGTGSEQDYLQACRAWLAYALDQYRNSGVEIVYTPPRYSIGVSRLQSGFKKLVSPSSAVEKCIVSVSYRGNNLNGLWLGDTIYCPRHVLGKFSGDQWNDVLNLANNHEFEVTTQHGVTLNVVSRRLKGAVLILQTAVANAETPKYKFIKANCGDSFTIACAYGGTVVGLYPVTMRSNGTIRASFLAGACGSVGFNIEKGVVNFFYMHHLELPNALHTGTDLMGEFYGGYVDEEVAQRVPPDNLVTNNIVAWLYAAIISVKESSFSLPKWLESTTVSVDDYNKWAGDNGFTPFSTSTAITKLSAITGVDVCKLLRTIMVKNSQWGGDPILGQYNFEDELTPESVFNQIGGVRLQSSFVRKATSWFWSRCVLACFLFVLCAIVLFTAVPLKFYVYAAVILLMAVLFISFTVKHVMAYMDTFLLPTLITVIIGVCAEVPFIYNTLISQVVIFLSQWYDPVVFDTMVPWMFLPLVLYTAFKCVQGCYMNSFNTSLLMLYQFVKLGFVIYTSSNTLTAYTEGNWELFFELVHTTVLANVSSNSLIGLFVFKCAKWMLYYCNATYLNNYVLMAVMVNCIGWLCTCYFGLYWWVNKVFGLTLGKYNFKVSVDQYRYMCLHKINPPKTVWEVFSTNILIQGIGGDRVLPIATVQAKLSDVKCTTVVLMQLLTKLNVEANSKMHVYLVELHNKILASDDVGECMDNLLGMLITLFCIDSTIDLSEYCDDILKRSTVLQSVTQEFSHIPSYAEYERAKNLYEKVLVDSKNGGVTQQELAAYRKAANIAKSVFDRDLAVQKKLDSMAERAMTTMYKEARVTDRRAKLVSSLHALLFSMLKKIDSEKLNVLFDQASSGVVPLATVPIVCSNKLTLVIPDPETWVKCVEGVHVTYSTVVWNIDTVIDADGTELHPTSTGSGLTYCISGANIAWPLKVNLTRNGHNKVDVVLQNNELMPHGVKTKACVAGVDQAHCSVESKCYYTNISGNSVVAAITSSNPNLKVASFLNEAGNQIYVDLDPPCKFGMKVGVKVEVVYLYFIKNTRSIVRGMVLGAISNVVVLQSKGHETEEVDAVGILSLCSFAVDPADTYCKYVAAGNQPLGNCVKMLTVHNGSGFAITSKPSPTPDQDSYGGASVCLYCRAHIAHPGSVGNLDGRCQFKGSFVQIPTTEKDPVGFCLRNKVCTVCQCWIGYGCQCDSLRQPKSSVQSVAGASDFDKNYLNRVRGSSEARLIPLASGCDPDVVKRAFDVCNKESAGMFQNLKRNCARFQELRDTEDGNLEYLDSYFVVKQTTPSNYEHEKSCYEDLKSEVTADHDFFVFNKNIYNISRQRLTKYTMMDFCYALRHFDPKDCEVLKEILVTYGCIEDYHPKWFEENKDWYDPIENSKYYVMLAKMGPIVRRALLNAIEFGNLMVEKGYVGVITLDNQDLNGKFYDFGDFQKTAPGAGVPVFDTYYSYMMPIIAMTDALAPERYFEYDVHKGYKSYDLLKYDYTEEKQELFQKYFKYWDQEYHPNCRDCSDDRCLIHCANFNILFSTLIPQTSFGNLCRKVFVDGVPFIATCGYHSKELGVIMNQDNTMSFSKMGLSQLMQFVGDPALLVGTSNNLVDLRTSCFSVCALTSGITHQTVKPGHFNKDFYDFAEKAGMFKEGSSIPLKHFFYPQTGNAAINDYDYYRYNRPTMFDICQLLFCLEVTSKYFECYEGGCIPASQVVVNNLDKSAGYPFNKFGKARLYYEMSLEEQDQLFEITKKNVLPTITQMNLKYAISAKNRARTVAGVSILSTMTNRQFHQKILKSIVNTRNASVVIGTTKFYGGWDNMLRNLIQGVEDPILMGWDYPKCDRAMPNLLRIAASLVLARKHTNCCSWSERIYRLYNECAQVLSETVLATGGIYVKPGGTSSGDATTAYANSVFNIIQATSANVARLLSVITRDIVYDNIKSLQYELYQQVYRRVNFDPAFVEKFYSYLCKNFSLMILSDDGVVCYNNTLAKQGLVADISGFREVLYYQNNVFMADSKCWVEPDLEKGPHEFCSQHTMLVEVDGEPKYLPYPDPSRILGACVFVDDVDKTEPVAVMERYIALAIDAYPLVHHENEEYKKVFFVLLAYIRKLYQELSQNMLMDYSFVMDIDKGSKFWEQEFYENMYRAPTTLQSCGVCVVCNSQTILRCGNCIRKPFLCCKCCYDHVMHTDHKNVLSINPYICSQLGCGEADVTKLYLGGMSYFCGNHKPKLSIPLVSNGTVFGIYRANCAGSENVDDFNQLATTNWSIVEPYILANRCSDSLRRFAAETVKATEELHKQQFASAEVREVFSDRELILSWEPGKTRPPLNRNYVFTGYHFTRTSKVQLGDFTFEKGEGKDVVYYKATSTAKLSVGDIFVLTSHNVVSLVAPTLCPQQTFSRFVNLRPNVMVPECFVNNIPLYHLVGKQKRTTVQGPPGSGKSHFAIGLAVYFSSARVVFTACSHAAVDALCEKAFKFLKVDDCTRIVPQRTTVDCFSKFKANDTGKKYIFSTINALPEVSCDILLVDEVSMLTNYELSFINGKINYQYVVYVGDPAQLPAPRTLLNGSLSPKDYNVVTNLMVCVKPDIFLAKCYRCPKEIVDTVSTLVYDGKFIANNPESRECFKVIVNNGNSDVGHESGSAYNTTQLEFVKDFVCRNKQWREAIFISPYNAMNQRAYRMLGLNVQTVDSSQGSEYDYVIFCVTADSQHALNINRFNVALTRAKRGILVVMRQRDELYSALKFTELDSETSLQGTGLFKICNKEFSGVHPAYAVTTKALAATYKVNDELAALVNVEAGSEITYKHLISLLGFKMSVNVEGCHNMFITRDEAIRNVRGWVGFDVEATHACGTNIGTNLPFQVGFSTGADFVVTPEGLVDTSIGNNFEPVNSKAPPGEQFNHLRVLFKSAKPWHVIRPRIVQMLADNLCNVSDCVVFVTWCHGLELTTLRYFVKIGKEQVCSCGSRATTFNSHTQAYACWKHCLGFDFVYNPLLVDIQQWGYSGNLQFNHDLHCNVHGHAHVASVDAIMTRCLAINNAFCQDVNWDLTYPHIANEDEVNSSCRYLQRMYLNACVDALKVNVVYDIGNPKGIKCVRRGDVNFRFYDKNPIVRNVKQFEYDYNQHKDKFADGLCMFWNCNVDCYPDNSLVCRYDTRNLSVFNLPGCNGGSLYVNKHAFYTPKFDRISFRNLKAMPFFFYDSSPCETIQVDGVAQDLVSLATKDCITKCNIGGAVCKKHAQMYAEFVTSYNAAVTAGFTFWVTNKLNPYNLWKSFSALQSIDNIAYNMYKGGHYDAIAGEMPTVITGDKVFVIDQGVEKAVFVNQTTLPTSVAFELYAKRNIRTLPNNRILKGLGVDVTNGFVIWDYANQTPLYRNTVKVCAYTDIEPNGLVVLYDDRYGDYQSFLAADNAVLVSTQCYKRYSYVEIPSNLLVQNGMPLKDGANLYVYKRVNGAFVTLPNTINTQGRSYETFEPRSDIERDFLAMSEESFVERYGKDLGLQHILYGEVDKPQLGGLHTVIGMYRLLRANKLNAKSVTNSDSDVMQNYFVLSDNGSYKQVCTVVDLLLDDFLELLRNILKEYGTNKSKVVTVSIDYHSINFMTWFEDGSIKTCYPQLQSAWTCGYNMPELYKVQNCVMEPCNIPNYGVGITLPSGILMNVAKYTQLCQYLSKTTICVPHNMRVMHFGAGSDKGVAPGSTVLKQWLPEGTLLVDNDIVDYVSDAHVSVLSDCNKYNTEHKFDLVISDMYTDNDSKRKHEGVIANNGNDDVFIYLSSFLRNNLALGGSFAVKVTETSWHEVLYDIAQDCAWWTMFCTAVNASSSEAFLIGVNYLGASEKVKVSGKTLHANYIFWRNCNYLQTSAYSIFDVAKFDLRLKATPVVNLKTEQKTDLVFNLIKCGKLLVRDVGNTSFTSDSFVCTM"""
_hide_fasta:str = """>reference
MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEECDTNSECEEEDEDTKVLALIQDPASIKYPLPLDEDYSVYNGCIVHKDALDVVNLPSGEETFVVNNCFEGAVKPLPQKVVDVLGDWGEAVDAQEQLCQQEPLQHTFEEPVENSTGSSKTMTEQVVVEDQELPVVEQDQDVVVYTPTDLEVAKETAEEVDEFILIFAVPKEEVVSQKDGAQIKQEPIQVVKPQREKKAKKFKVKPATCEKPKFLEYKTCVGDLTVVIAKALDEFKEFCIVNAANEHMTHGSGVAKAIADFCGLDFVEYCEDYVKKHGPQQRLVTPSFVKGIQCVNNVVGPRHGDNNLHEKLVAAYKNVLVDGVVNYVVPVLSLGIFGVDFKMSIDAMREAFEGCTIRVLLFSLSQEHIDYFDVTCKQKTIYLTEDGVKYRSIVLKPGDSLGQFGQVYAKNKIVFTADDVEDKEILYVPTTDKSILEYYGLDAQKYVIYLQTLAQKWNVQYRDNFLILEWRDGNCWISSAIVLLQAAKIRFKGFLTEAWAKLLGGDPTDFVAWCYASCTAKVGDFSDANWLLANLAEHFDADYTNAFLKKRVSCNCGIKSYELRGLEACIQPVRATNLLHFKTQYSNCPTCGANNTDEVIEASLPYLLLFATDGPATVDCDEDAVGTVVFVGSTNSGHCYTQAAGQAFDNLAKDRKFGKKSPYITAMYTRFAFKNETSLPVAKQSKGKSKSVKEDVSNLATSSKASFDNLTDFEQWYDSNIYESLKVQESPDNFDKYVSFTTKEDSKLPLTLKVRGIKSVVDFRSKDGFIYKLTPDTDENSKAPVYYPVLDAISLKAIWVEGNANFVVGHPNYYSKSLHIPTFWENAENFVKMGDKIGGVTMGLWRAEHLNKPNLERIFNIAKKAIVGSSVVTTQCGKLIGKAATFIADKVGGGVVRNITDSIKGLCGITRGHFERKMSPQFLKTLMFFLFYFLKASVKSVVASYKTVLCKVVLATLLIVWFVYTSNPVMFTGIRVLDFLFEGSLCGPYKDYGKDSFDVLRYCADDFICRVCLHDKDSLHLYKHAYSVEQVYKDAASGFIFNWNWLYLVFLILFVKPVAGFVIICYCVKYLVLNSTVLQTGVCFLDWFVQTVFSHFNFMGAGFYFWLFYKIYIQVHHILYCKDVTCEVCKRVARSNRQEVSVVVGGRKQIVHVYTNSGYNFCKRHNWYCRNCDDYGHQNTFMSPEVAGELSEKLKRHVKPTAYAYHVVDEACLVDDFVNLKYKAATPGKDSASSAVKCFSVTDFLKKAVFLKEALKCEQISNDGFIVCNTQSAHALEEAKNAAIYYAQYLCKPILILDQALYEQLVVEPVSKSVIDKVCSILSSIISVDTAALNYKAGTLRDALLSITKDEEAVDMAIFCHNHDVDYTGDGFTNVIPSYGIDTGKLTPRDRGFLINADASIANLRVKNAPPVVWKFSELIKLSDSCLKYLISATVKSGVRFFITKSGAKQVIACHTQKLLVEKKAGGIVSGTFKCFKSYFKWLLIFYILFTACCSGYYYMEVSKSFVHPMYDVNSTLHVEGFKVIDKGVLREIVPEDTCFSNKFVNFDAFWGRPYDNSRNCPIVTAVIDGDGTVATGVPGFVSWVMDGVMFIHMTQTERKPWYIPTWFNREIVGYTQDSIITEGSFYTSIALFSARCLYLTASNTPQLYCFNGDNDAPGALPFGSIIPHRVYFQPNGVRLIVPQQILHTPYVVKFVSDSYCRGSVCEYTRPGYCVSLNPQWVLFNDEYTSKPGVFCGSTVRELMFSMVSTFFTGVNPNIYMQLATMFLILVVVVLIFAMVIKFQGVFKAYATTVFITMLVWVINAFILCVHSYNSVLAVILLVLYCYASLVTSRNTVIIMHCWLVFTFGLIVPTWLACCYLGFIIYMYTPLFLWCYGTTKNTRKLYDGNEFVGNYDLAAKSTFVIRGSEFVKLTNEIGDKFEAYLSAYARLKYYSGTGSEQDYLQACRAWLAYALDQYRNSGVEIVYTPPRYSIGVSRLQSGFKKLVSPSSAVEKCIVSVSYRGNNLNGLWLGDTIYCPRHVLGKFSGDQWNDVLNLANNHEFEVTTQHGVTLNVVSRRLKGAVLILQTAVANAETPKYKFIKANCGDSFTIACAYGGTVVGLYPVTMRSNGTIRASFLAGACGSVGFNIEKGVVNFFYMHHLELPNALHTGTDLMGEFYGGYVDEEVAQRVPPDNLVTNNIVAWLYAAIISVKESSFSLPKWLESTTVSVDDYNKWAGDNGFTPFSTSTAITKLSAITGVDVCKLLRTIMVKNSQWGGDPILGQYNFEDELTPESVFNQIGGVRLQSSFVRKATSWFWSRCVLACFLFVLCAIVLFTAVPLKFYVYAAVILLMAVLFISFTVKHVMAYMDTFLLPTLITVIIGVCAEVPFIYNTLISQVVIFLSQWYDPVVFDTMVPWMFLPLVLYTAFKCVQGCYMNSFNTSLLMLYQFVKLGFVIYTSSNTLTAYTEGNWELFFELVHTTVLANVSSNSLIGLFVFKCAKWMLYYCNATYLNNYVLMAVMVNCIGWLCTCYFGLYWWVNKVFGLTLGKYNFKVSVDQYRYMCLHKINPPKTVWEVFSTNILIQGIGGDRVLPIATVQAKLSDVKCTTVVLMQLLTKLNVEANSKMHVYLVELHNKILASDDVGECMDNLLGMLITLFCIDSTIDLSEYCDDILKRSTVLQSVTQEFSHIPSYAEYERAKNLYEKVLVDSKNGGVTQQELAAYRKAANIAKSVFDRDLAVQKKLDSMAERAMTTMYKEARVTDRRAKLVSSLHALLFSMLKKIDSEKLNVLFDQASSGVVPLATVPIVCSNKLTLVIPDPETWVKCVEGVHVTYSTVVWNIDTVIDADGTELHPTSTGSGLTYCISGANIAWPLKVNLTRNGHNKVDVVLQNNELMPHGVKTKACVAGVDQAHCSVESKCYYTNISGNSVVAAITSSNPNLKVASFLNEAGNQIYVDLDPPCKFGMKVGVKVEVVYLYFIKNTRSIVRGMVLGAISNVVVLQSKGHETEEVDAVGILSLCSFAVDPADTYCKYVAAGNQPLGNCVKMLTVHNGSGFAITSKPSPTPDQDSYGGASVCLYCRAHIAHPGSVGNLDGRCQFKGSFVQIPTTEKDPVGFCLRNKVCTVCQCWIGYGCQCDSLRQPKSSVQSVAGASDFDKNYLNRVRGSSEARLIPLASGCDPDVVKRAFDVCNKESAGMFQNLKRNCARFQELRDTEDGNLEYLDSYFVVKQTTPSNYEHEKSCYEDLKSEVTADHDFFVFNKNIYNISRQRLTKYTMMDFCYALRHFDPKDCEVLKEILVTYGCIEDYHPKWFEENKDWYDPIENSKYYVMLAKMGPIVRRALLNAIEFGNLMVEKGYVGVITLDNQDLNGKFYDFGDFQKTAPGAGVPVFDTYYSYMMPIIAMTDALAPERYFEYDVHKGYKSYDLLKYDYTEEKQELFQKYFKYWDQEYHPNCRDCSDDRCLIHCANFNILFSTLIPQTSFGNLCRKVFVDGVPFIATCGYHSKELGVIMNQDNTMSFSKMGLSQLMQFVGDPALLVGTSNNLVDLRTSCFSVCALTSGITHQTVKPGHFNKDFYDFAEKAGMFKEGSSIPLKHFFYPQTGNAAINDYDYYRYNRPTMFDICQLLFCLEVTSKYFECYEGGCIPASQVVVNNLDKSAGYPFNKFGKARLYYEMSLEEQDQLFEITKKNVLPTITQMNLKYAISAKNRARTVAGVSILSTMTNRQFHQKILKSIVNTRNASVVIGTTKFYGGWDNMLRNLIQGVEDPILMGWDYPKCDRAMPNLLRIAASLVLARKHTNCCSWSERIYRLYNECAQVLSETVLATGGIYVKPGGTSSGDATTAYANSVFNIIQATSANVARLLSVITRDIVYDNIKSLQYELYQQVYRRVNFDPAFVEKFYSYLCKNFSLMILSDDGVVCYNNTLAKQGLVADISGFREVLYYQNNVFMADSKCWVEPDLEKGPHEFCSQHTMLVEVDGEPKYLPYPDPSRILGACVFVDDVDKTEPVAVMERYIALAIDAYPLVHHENEEYKKVFFVLLAYIRKLYQELSQNMLMDYSFVMDIDKGSKFWEQEFYENMYRAPTTLQSCGVCVVCNSQTILRCGNCIRKPFLCCKCCYDHVMHTDHKNVLSINPYICSQLGCGEADVTKLYLGGMSYFCGNHKPKLSIPLVSNGTVFGIYRANCAGSENVDDFNQLATTNWSIVEPYILANRCSDSLRRFAAETVKATEELHKQQFASAEVREVFSDRELILSWEPGKTRPPLNRNYVFTGYHFTRTSKVQLGDFTFEKGEGKDVVYYKATSTAKLSVGDIFVLTSHNVVSLVAPTLCPQQTFSRFVNLRPNVMVPECFVNNIPLYHLVGKQKRTTVQGPPGSGKSHFAIGLAVYFSSARVVFTACSHAAVDALCEKAFKFLKVDDCTRIVPQRTTVDCFSKFKANDTGKKYIFSTINALPEVSCDILLVDEVSMLTNYELSFINGKINYQYVVYVGDPAQLPAPRTLLNGSLSPKDYNVVTNLMVCVKPDIFLAKCYRCPKEIVDTVSTLVYDGKFIANNPESRECFKVIVNNGNSDVGHESGSAYNTTQLEFVKDFVCRNKQWREAIFISPYNAMNQRAYRMLGLNVQTVDSSQGSEYDYVIFCVTADSQHALNINRFNVALTRAKRGILVVMRQRDELYSALKFTELDSETSLQGTGLFKICNKEFSGVHPAYAVTTKALAATYKVNDELAALVNVEAGSEITYKHLISLLGFKMSVNVEGCHNMFITRDEAIRNVRGWVGFDVEATHACGTNIGTNLPFQVGFSTGADFVVTPEGLVDTSIGNNFEPVNSKAPPGEQFNHLRVLFKSAKPWHVIRPRIVQMLADNLCNVSDCVVFVTWCHGLELTTLRYFVKIGKEQVCSCGSRATTFNSHTQAYACWKHCLGFDFVYNPLLVDIQQWGYSGNLQFNHDLHCNVHGHAHVASVDAIMTRCLAINNAFCQDVNWDLTYPHIANEDEVNSSCRYLQRMYLNACVDALKVNVVYDIGNPKGIKCVRRGDVNFRFYDKNPIVRNVKQFEYDYNQHKDKFADGLCMFWNCNVDCYPDNSLVCRYDTRNLSVFNLPGCNGGSLYVNKHAFYTPKFDRISFRNLKAMPFFFYDSSPCETIQVDGVAQDLVSLATKDCITKCNIGGAVCKKHAQMYAEFVTSYNAAVTAGFTFWVTNKLNPYNLWKSFSALQSIDNIAYNMYKGGHYDAIAGEMPTVITGDKVFVIDQGVEKAVFVNQTTLPTSVAFELYAKRNIRTLPNNRILKGLGVDVTNGFVIWDYANQTPLYRNTVKVCAYTDIEPNGLVVLYDDRYGDYQSFLAADNAVLVSTQCYKRYSYVEIPSNLLVQNGMPLKDGANLYVYKRVNGAFVTLPNTINTQGRSYETFEPRSDIERDFLAMSEESFVERYGKDLGLQHILYGEVDKPQLGGLHTVIGMYRLLRANKLNAKSVTNSDSDVMQNYFVLSDNGSYKQVCTVVDLLLDDFLELLRNILKEYGTNKSKVVTVSIDYHSINFMTWFEDGSIKTCYPQLQSAWTCGYNMPELYKVQNCVMEPCNIPNYGVGITLPSGILMNVAKYTQLCQYLSKTTICVPHNMRVMHFGAGSDKGVAPGSTVLKQWLPEGTLLVDNDIVDYVSDAHVSVLSDCNKYNTEHKFDLVISDMYTDNDSKRKHEGVIANNGNDDVFIYLSSFLRNNLALGGSFAVKVTETSWHEVLYDIAQDCAWWTMFCTAVNASSSEAFLIGVNYLGASEKVKVSGKTLHANYIFWRNCNYLQTSAYSIFDVAKFDLRLKATPVVNLKTEQKTDLVFNLIKCGKLLVRDVGNTSFTSDSFVCTM"""

In [ ]:
### Parse muscle outfile
from Bio import SeqIO

outfile_path:str = "./outfile.fasta"
out:dict = {}
for record in SeqIO.parse(outfile_path, 'fasta'):
    out[record.id] = record.seq


In [ ]:
### detect events -> variant calling
events:list = []
# -> ('origin', 'pos', 'mut')

for k in range(len(out['reference'])):
    r = out['reference'][k]
    q = out['query'][k]
    if r != q:
        events.append((r,k,q))
print(events)


In [ ]:
### PP and nsps

import json, os

testydata: str = "./../../../testymolo/media/data/"
Proteins:list = json.load(open(os.path.join(testydata,'Protein.temp.json')))
data_ac:int = 473

PP = [ pp for pp in Proteins if (pp['isPP'] and (pp['data_ac'] == data_ac)) ][0]
nsps = [ nsp for nsp in Proteins if (nsp['derivedFromPP'] and (nsp['data_ac'] == data_ac)) ]

In [ ]:
Polyproteins:list = json.load(open(os.path.join(testydata,'PolyProtein.temp.json')))

for nsp in nsps:
    pp_link = [pp for pp in Polyproteins if pp['protein']==nsp['id']][0]
    nsp['start'] = pp_link['start']
    nsp['index'] = pp_link['index']

In [ ]:
print(PP['sequence'])
for nsp in nsps:
    print(' '*(nsp['start']-1) + nsp['sequence'] )

In [ ]:
modified_sequence = PP['sequence']
for e in events:
    r,k,q = e
    modified_sequence = modified_sequence[:k] + q + modified_sequence[k+1:]

print(modified_sequence)
for nsp in nsps:
    print(' '*(nsp['start']-1) + nsp['sequence'] )

In [ ]:
### impact on subseqs

import json, os

testydata: str = "./../../../testymolo/media/data/"
Subseqs:list = json.load(open(os.path.join(testydata,'Subseq.temp.json')))

subseq:list = list([ sseq for sseq in Subseqs if sseq['origin'] == 1 ])

print(subseq)

In [ ]:
from Bio import Entrez

Entrez.email = "VIMVer@univ-amu.fr"

handle = Entrez.esearch(db='genome', retmax=300, term="txid76804[Organism:exp]", idtype='acc')
record = Entrez.read(handle)
handle.close()
print(len(record['IdList']))

In [ ]:
handle = Entrez.esearch(db='nucleotide', retmax=300, term="txid76804[Organism:exp] AND refseq[filter]", idtype='acc')
records = Entrez.read(handle)
handle.close()
print(len(records['IdList']))

In [ ]:
from Bio import SeqIO

lengths:list = []

for acc in records['IdList']:
    #print(acc)
    efetch = Entrez.efetch(db='nucleotide', id=acc, rettype='gb', retmode='text')
    record = SeqIO.parse(efetch, 'gb')
    for item in record:
        lengths.append(len(str(item.seq)))

In [ ]:
efetch = Entrez.efetch(db='nucleotide', id="YP_009047202", rettype='gb', retmode='text')
record = SeqIO.parse(efetch, 'gb')
print(record.id)

In [ ]:
print(len(lengths))
print(min(lengths))
print(int(sum(lengths)/float(len(lengths))))
print(max(lengths))


In [61]:
from Bio import SeqIO
import os
import json


def from_genbank_to_dict_for_adding_to_model(acc:str):
    outdata:dict = {}
    outdata['p'] = 0
    outdata['Organism'] = {}
    outdata['Protein'] = []
    outdata['Polyprotein'] = []

    lexico:dict = {
        'Wuhan-Hu-1': 'SARS-CoV-2',
        '3C-like ase':'nsp5',
        '3C-like proteinase': 'nsp5',
        'ORF': 'pp',
        'RNA-dependent RNA polymerase': 'nsp12',
        'helicase': 'nsp13',
        'Hel':'nsp13',
        "3'-to-5' exonuclease": 'nsp14',
        "ExoN": 'nsp14',
        "endoRNAse": 'nsp15',
        "NendoU":'nsp15',
        "2'-O-ribose methyltransferase": 'nsp16',
        "2'-O-methyltransferase":'nsp16'
    }
        
    with open(os.path.join("./genbanks",acc), 'r') as handle:
        for record in SeqIO.parse(handle, 'gb'):
            print(record.id)
            name:str = record.description
            name = name.split('[')[0].strip()
            if 'polyprotein' in name:
                name.replace('polyprotein', '').strip()
            outdata['Protein'].append({
                'isPP': True, 'derivedFromPP': False,
                'genbank': record.id, 'name': name,
                'header': f"{name.replace(' ','_')}__(@{record.id})",
                'sequence': str(record.seq)
            })
            outdata['p'] = 0

            for feat in record.features:
                if feat.type == 'source':
                    abr:str = feat.qualifiers['isolate'][0]
                    if abr in lexico:
                        abr = lexico[abr]
                    outdata['Organism'] = {
                        'name': feat.qualifiers['organism'][0],
                        'id': feat.qualifiers['db_xref'][0].split(':')[-1],
                        'abr': abr,
                        'phylogeny': '; '.join(record.annotations['taxonomy'])
                    }
                    
                if feat.type == 'mat_peptide':
                    boundaries = ( int(feat.location.start) , int(feat.location.end) )
                    #print(boundaries)
                    name:str = feat.qualifiers['product'][0]
                    if 'protein' in name:
                        name = name.replace('protein', '').strip()
                    if name in lexico:
                        name = lexico[name]
                    prot_id = feat.qualifiers['protein_id'][0]
                    outdata['Protein'].append({
                        'isPP': False, 'derivedFromPP':True, 
                        'genbank': feat.qualifiers['protein_id'][0],
                        'name': name,
                        'header': f"{name.replace(' ','_')}__(@{prot_id})",
                        'sequence': outdata['Protein'][0]['sequence'][boundaries[0]: boundaries[1]]
                    })
                    outdata['p'] += 1
                    outdata['Polyprotein'].append({
                        'PP': 0 , 'protein': outdata['p'], 'index': outdata['p'], 
                        'start': boundaries[0]+1, 'end': boundaries[1] })

    with open(f"./outdata/{acc}.json", 'w') as out:
        out.write(json.dumps(outdata))

    

In [62]:
print(os.listdir("./genbanks/"))

['MERS_pp1ab.gp', 'SARS-CoV-2.pp1a.gp', 'MERS.pp1a.gp', 'SARS-CoV-2_pp1ab.gp']


In [63]:
from_genbank_to_dict_for_adding_to_model("MERS_pp1ab.gp")

YP_009047202.1
